# 4. Inference

In [1]:
import sys, os
import torch
import pandas as pd
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.tool import clean as ml_clean
import pathSetting

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "regression"

### 4-1. Get model meta by mongodb

In [3]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins

mongo_client = MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'TestGRU_rg'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[-3]

In [5]:
model_meta

{'trainDataInfo': {'startTime': '2016-01-11',
  'endTime': '2016-04-15',
  'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 600,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'modelName': 'TestGRU_rg',
 'featureList': ['RH_1',
  'RH_2',
  'RH_3',
  'RH_4',
  'RH_5',
  'RH_6',
  'RH_7',
  'RH_8',
  'RH_9',
  'T1',
  'T2',
  'T3',
  'T4',
  'T5',
  'T6',
  'T7',
  'T8',
  'T9',
  'Press_mm_hg',
  'RH_out',
  'T_out',
  'Tdewpoint',
  'Visibility',
  'Windspeed'],
 '

### 4-2. Inference data selection

In [6]:
# 0. pick only one data
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [7]:
dataList

['IntegraionTrainX', 'IntegraionTrainy', 'IntegraionTestX', 'IntegraionTesty']

In [8]:
if mode_selection == 'regression':
    # dataX

    dataName_X = dataList[2]
    dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

    dataFolderName = "data_integrated_result"
    current = os.getcwd()
    dataFolderPath = os.path.join(current, dataFolderName)

    window_num =144
    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]

elif mode_selection == 'forecast':
    LearningModeList = ['train', 'test']
    LearningMode = LearningModeList[1]

    cleanParamList = ['Clean', 'NoClean']
    cleanMode = cleanParamList[1]

    datasetNameList = ['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']
    datasetName = datasetNameList[0]

    dataName_X = LearningMode + cleanMode + '_' + datasetName
    dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, pathSetting.dataFolderPath)
    integration_freq_sec = DataMeta[dataName_X]['integrationInfo']['integration_freq_sec']

In [9]:
print(dataX)

                                RH_1       RH_2       RH_3       RH_4  \
datetime                                                                
2021-01-01 00:00:00+00:00  40.260000  43.200000  38.530000  37.363333   
2021-01-01 00:10:00+00:00  40.260000  43.163333  38.500000  37.230000   
2021-01-01 00:20:00+00:00  40.290000  42.963333  38.633333  37.200000   
2021-01-01 00:30:00+00:00  40.290000  42.490000  38.790000  37.200000   
2021-01-01 00:40:00+00:00  40.626667  42.156667  38.596667  37.090000   
...                              ...        ...        ...        ...   
2021-01-01 23:10:00+00:00  41.500000  42.090000  38.290000  40.730000   
2021-01-01 23:20:00+00:00  41.833333  42.290000  38.290000  40.790000   
2021-01-01 23:30:00+00:00  42.090000  42.363333  38.290000  41.166667   
2021-01-01 23:40:00+00:00  42.030000  42.433333  38.363333  42.093333   
2021-01-01 23:50:00+00:00  41.900000  42.500000  38.290000  42.400000   

                                RH_5       RH_6   

### 4-3. Inference data preparation

In [10]:
feature_list = model_meta['featureList']
target = model_meta['target']
scaler_param = model_meta['scalerParam']
transform_param = model_meta["transformParameter"]
model_file_path = model_meta['files']['modelFile']['filePath']
model_method = model_meta['model_method']
# train_parameter = model_meta["trainParameter"]

if mode_selection == 'regression':
    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
    y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]

    infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

elif mode_selection == 'forecast':
    clean_param = model_meta['cleanTrainDataParam']
    nan_processing_param = model_meta['NaNProcessingParam']

    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']['filePath']

    # only for forecast data?
    past_step = transform_param['past_step']
    feature_data = dataX[feature_list]
    step_data = feature_data[-past_step:][feature_list].values
    df_data = pd.DataFrame(step_data, columns=feature_list)

    infer_X, scaler_X = ml_scaler.get_scaled_test_data(df_data[feature_list], X_scaler_file_path, scaler_param)
    clean_infer_X = ml_clean.get_cleand_data(infer_X, clean_param, integration_freq_sec, nan_processing_param)

In [11]:
print(infer_X)

                               RH_1      RH_2      RH_3      RH_4      RH_5  \
datetime                                                                      
2021-01-01 00:00:00+00:00  0.364279  0.664944  0.456302  0.386490  0.275336   
2021-01-01 00:10:00+00:00  0.364279  0.663872  0.454900  0.380531  0.275336   
2021-01-01 00:20:00+00:00  0.365104  0.658023  0.461131  0.379190  0.274333   
2021-01-01 00:30:00+00:00  0.365104  0.644180  0.468453  0.379190  0.274333   
2021-01-01 00:40:00+00:00  0.374369  0.634432  0.459417  0.374274  0.274333   
...                             ...       ...       ...       ...       ...   
2021-01-01 23:10:00+00:00  0.398404  0.632482  0.445085  0.536963  0.335480   
2021-01-01 23:20:00+00:00  0.407577  0.638331  0.445085  0.539644  0.331521   
2021-01-01 23:30:00+00:00  0.414641  0.640476  0.445085  0.556479  0.328413   
2021-01-01 23:40:00+00:00  0.412990  0.642523  0.448512  0.597896  0.326960   
2021-01-01 23:50:00+00:00  0.409412  0.644473  0.445

In [12]:
# transform
if mode_selection == 'regression':

    from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
    inferX = trans_df_to_np_inf(infer_X)

# forecast
elif mode_selection == 'forecast':
    
    from Clust.clust.transformation.purpose.machineLearning import LSTMData
    LSTMD = LSTMData()
    testX, testy = LSTMD.transform_Xy_arr(clean_infer_X, transformParameter, transformParameter['clean_param'])

/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:221: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]


In [16]:
print(type(inferX), inferX.shape)
print(inferX)

<class 'numpy.ndarray'> (1, 24, 144)
[[[0.36427851 0.36427851 0.36510412 ... 0.41464086 0.41298963 0.40941198]
  [0.66494443 0.6638721  0.65802301 ... 0.64047573 0.64252291 0.64447261]
  [0.4563016  0.45489952 0.46113102 ... 0.4450849  0.44851223 0.4450849 ]
  ...
  [0.48868778 0.47963801 0.47058824 ... 0.67420814 0.67571644 0.67722474]
  [0.609375   0.609375   0.609375   ... 0.609375   0.609375   0.609375  ]
  [0.30769231 0.29487179 0.28205128 ... 0.53846154 0.51282051 0.48717949]]]


In [14]:
modelParameter = model_meta["modelParameter"]

inferParameter = {
    'device': 'cpu',
    'batch_size': 16
}

In [15]:
# 4. Inference
from Clust.clust.ML.regression_YK.inference import RegressionInference as RI

ri = RI()
ri.set_param(inferParameter)
ri.set_model(model_method, model_file_path, modelParameter)
ri.set_data(infer_X)
preds = ri.inference()
print(preds)

ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
# for regression
if mode_selection == 'regression':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
    else:
        prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

# for forecast
elif mode_selection == 'forecast':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_list, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        inverse_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
        target_data = inverse_result[target]
        prediction_result = pd.DataFrame(data={target: target_data}, index=range(len(preds)))
    else:
        prediction_result = pd.DataFrame(data={target: preds}, index=range(len(preds)))

ValueError: Columns must be same length as key

In [ ]:
prediction_result

,value
0,9.986181
